In [60]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, TimestampType
from pyspark.sql import DataFrameWriter
from pyspark.sql.functions import  regexp_extract, col, regexp_replace
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

In [61]:
socketDF = spark.readStream.format("socket").option("host", "seppe.net").option("port", 7778).load()

socketDF.printSchema()


root
 |-- value: string (nullable = true)



In [62]:
from pyspark.sql.functions import from_json, schema_of_json

In [63]:
def process_row(df, epoch_id):
    print(epoch_id)
    if df.count() == 0: return
    schema = schema_of_json(df.first().value)
    df_cols = df.selectExpr('CAST(value AS STRING)')\
        .select(from_json('value', schema)\
        .alias('temp'))\
        .select('temp.*')
    df_cols.show()

In [69]:


test = socketDF.writeStream\
    .format("parquet")\
    .option("path", "file:///C:/Users/ignac/Documents/data/deel3/tweets")\
    .option("checkpointLocation", "file:///C:/Users/ignac/Documents/data/deel3")\
    .start()



In [8]:
test.stop()

In [87]:



file_path1 = "file:///C:/Users/ignac/Documents/data/deel2/tweets"
file_path2 = "file:///C:/Users/ignac/Documents/data/tweets"
file_path3 = "file:///C:/Users/ignac/Documents/data/deel3/tweets"
rawDF1 = spark.read.parquet(file_path1)
rawDF2 = spark.read.parquet(file_path2)
rawDF3 = spark.read.parquet(file_path3)
print(rawDF1.count())
print(rawDF2.count())
print(rawDF3.count())


rawDF1_0 = rawDF1.union(rawDF2)
rawDF = rawDF1_0.union(rawDF3)

print(rawDF.count())

1118
82
829
2029


In [88]:

regex1 = r'(\d+)'
regex2 = r'("#\w+")'
regex3 = r'"tweet_id": \d+, "tweet_text": ' 
regex4 = r'"label": "#\w+"'
DF = rawDF.withColumn('tweet_id',regexp_extract(col('value'), regex1, 1))\
.withColumn('label',regexp_extract(col('value'), regex2, 1))\
.withColumn('tweet_text1',F.regexp_replace('value', regex3, ''))

DF = DF.withColumn('tweet_text',F.regexp_replace('tweet_text1', regex4, ''))

DF.write.parquet('file:///C:/Users/ignac/Documents/data/dataframe2', mode="overwrite")

In [89]:
df = spark.read.load("file:///C:/Users/ignac/Documents/data/dataframe2")

df.show()

+--------------------+-------------------+----------------+--------------------+--------------------+
|               value|           tweet_id|           label|         tweet_text1|          tweet_text|
+--------------------+-------------------+----------------+--------------------+--------------------+
|{"tweet_id": 1389...|1389090870696185859|        "#biden"|{"@2ordinary1 @cl...|{"@2ordinary1 @cl...|
|{"tweet_id": 1388...|1388991974263463937|        "#china"|{"#\u2588\u2588\u...|{"#\u2588\u2588\u...|
|{"tweet_id": 1389...|1389083482320560130|        "#china"|{"TSMC says can c...|{"TSMC says can c...|
|{"tweet_id": 1389...|1389098504778403841|    "#inflation"|{"\ud83d\udea8\ud...|{"\ud83d\udea8\ud...|
|{"tweet_id": 1389...|1389079721766391813|        "#covid"|{"It was a heart,...|{"It was a heart,...|
|{"tweet_id": 1389...|1389053238268227584|        "#covid"|{"Our liberty dep...|{"Our liberty dep...|
|{"tweet_id": 1389...|1389069630895915013|      "#vaccine"|{"#\u2588\u2588\u...|{"

# testing (not relevant) 



In [ ]:
def process_row(df, epoch_id):
    print(epoch_id)
    if df.count() == 0: return
    schema = schema_of_json(df.first().value)
    df_cols = df.selectExpr('CAST(value AS STRING)')\
        .select(from_json('value', schema)\
        .alias('temp'))\
        .select('temp.*')
    df_cols.show()

In [ ]:
query = socketDF.writeStream\
    .trigger(processingTime='5 seconds')\
    .foreachBatch(process_row)\
    .option("path", "file:///C:/Users/ignac/Documents/data/tweets")\
    .option("checkpointLocation", "file:///C:/Users/ignac/Documents/data")\
    .start() 

In [ ]:
userSchema = StructType().add("tweet_id", "integer").add("tweet_text", "string").add("label", "string")

df = spark.readStream\
.format("parquet")\
.option("sep", ",")\
.schema(userSchema)\
.parquet("file:///C:/Users/ignac/Documents/data/tweets")